In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog_name}.udp_wcm_gold_vcm_db_inventory.d_mdq_store_dc_sku_daily_his
(
  calday DATE,
  business_unit STRING,
  store_id STRING,
  business_model STRING,
  mch2_id STRING,
  product_id STRING,
  manufacturer_id STRING,
  mdq FLOAT,
  external_report_mdq FLOAT,
  internal_report_mdq FLOAT,
  old_mdq FLOAT,
  mdq_age BIGINT,
  mdq_available STRING,
  dc_id STRING,
  replenishment_mode STRING,
  sku_type STRING,
  assortment_type STRING,
  is_problem_article STRING
)
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
""")

In [0]:
spark.sql(f"""
DELETE FROM {catalog_name}.udp_wcm_gold_vcm_db_inventory.d_mdq_store_dc_sku_daily_his WHERE CALDAY = DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS);
""")


In [0]:
spark.sql(f"""
INSERT INTO {catalog_name}.udp_wcm_gold_vcm_db_inventory.d_mdq_store_dc_sku_daily_his
SELECT
    DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) calday,
	business_unit,
    store_id,
    business_model,
    mch2_id,
    product_id,
    manufacturer_id,
    mdq,
    external_report_mdq,
    internal_report_mdq,
    old_mdq,
    mdq_age,
    mdq_available,
    dc_id,
    xd_dc_id,
    replenishment_mode,
    sku_type,
    assortment_type,
    is_problem_article
FROM {catalog_name}.udp_wcm_gold_vcm_db_inventory.d_mdq_store_dc_sku_daily
""")